<div style="background-color:#e6ffe6; padding:10px; border-style:
solid;; border-color:#00e600; border-width:1px">

# AST4310 2025, Project 2

</div>

# Background

In this project you will learn about radiation in stars and use different models of a stellar atmosphere, for two different stars. The exercises will build on the different ingredients to achieve the final goal: to calculate the predicted spectral lines from a model atmosphere of a star. This is a fundamental task to study any star in detail.

## 1. Julia code and imports

Before we can get started with the computional setup for this project, it is recommended you start your project with the following imports. More can be added if you need:

In [1]:
using Unitful
using CairoMakie
using JLD2
using Interpolations
using NumericalIntegration
import SpecialFunctions: gamma
import PhysicalConstants.CODATA2018: h, k_B, R_∞, c_0, m_e, m_u, e, ε_0, a_0, σ

In addition, several Julia functions and types are provided in the included file `tools.jl` for your convenience. These are mostly lengthy functions for recipes of continuum extinction or parts of line extinction, and we recommend you do not modify this file (if needed, you can redefine some functions in your notebook). You can add all these functions to your notebook by just including the file:

In [2]:
include("tools.jl")

const_barklem (generic function with 1 method)

You should put this include after your imports, as loading the data in JLD2 format depends on it.

## 2. Model Atmospheres

A model atmosphere is a basic building block to interpret radiation from stars. Different types of models exist, mostly in geometries of 1D plane-parallel, 1D spherical, or 3D (either cartesian or spherical). In this project you will work with a 3D cartesian model ran with the [Stagger code](https://ui.adsabs.harvard.edu/abs/2024ApJ...970...24S/abstract) code, and a 1D plane-parallel model of the Sun known as [FALC](https://ui.adsabs.harvard.edu/abs/1993ApJ...406..319F/abstract)

A model atmosphere consists of a bunch of thermodynamical variables such as temperature, density, velocity, etc. These are usually tabled in a *depth* scale, meaning that the first point in the z axis is the top of the atmosphere, and the last point is deeper in the star. Normally, together with an appropriate equation of state, quantities such as electron density, hydrogen density, etc. can be provided as well. 

### 2.1 3D model of a mystery star

<img src="https://i1.rgstatic.net/ii/profile.image/272391284719621-1441954551201_Q512/Robert_Stein.jpg" width="250px" alt="Bob Stein"> <img src="https://www.astro.ku.dk/~aake/aa03-med.jpg" style="width=250px; margin: -75px;" alt="Åke Nordlund">

*[Bob Stein](https://web.pa.msu.edu/people/steinr/research.html) (left, born 1935) and [Åke Nordlund](https://www.astro.ku.dk/~aake/)</a> (right, born 1947) are two pioneers of computer simulations of magneto-hydrodynamics (MHD). Their study of plasma constrained by magnetic fields have had a deep impact in various fields, from planetary formation to the very finest scales in stellar surfaces. In particular, their prolific work on convection in the atmospheres of cool stars has led to a paradigm shift from seeing stellar atmospheres as static 1D plane-parallel objects to highly-dynamic 3D atmospheres. Their seminal work [Simulations of Solar Granulation](https://ui.adsabs.harvard.edu/abs/1998ApJ...499..914S) triggered a new age in the study of the solar and stellar atmospheres, where spectral calculations from 3D MHD simulations of convection became feasible and challenged our understanding of stellar spectra. Using such simulations, the solar metallicity,  stable during decades of modelling and observations, was suddenly challenged and revised by almost half by [Asplund et al. (2005)](https://ui.adsabs.harvard.edu/abs/2005ASPC..336...25A). The codes developed by Nordlund and Stein have thrived and continued to be developed and updated at many research groups. They were at the origin of the Bifrost code, actively developed at the University of Oslo. Photographs from the web.*

1D plane-parallel models are horizontally homogeneous and in hydrostatic equilibrium ("time independent"). Of course, stellar atmospheres are neither homogeneous nor static. A more realistic description of a stellar atmosphere is given by time-dependent, radiative 3D magneto-hydrodynamic (MHD) simulations. They can capture more physics, such as a realistic treatment of convection and radiation at different depths, and don't need the fudge factors such as microturbulence to make the spectra look like the observations. 

A proper use of 3D models should include the temporal dimension, but for the sake of convenience and speedy calculations, we will use only a single timestep in this exercise. 

The 3D model file is too large to host on Github, so you need to [download it](https://www.uio.no/studier/emner/matnat/astro/AST4310/h25/files/mystery_star.jld2) (277 MB) and save it to the current directory. This is one timestep of a simulation run with the [Stagger code](https://ui.adsabs.harvard.edu/abs/2024ApJ...970...24S/abstract). This particular simulation was provided courtesy of Dr. Yixiao Zhou and has no magnetic fields and does not include the hotter layers of the transition region and corona (not relevant for this exercise). Here it is saved as a JLD2 file, which you can load it into an structure called `atm` with the following command:

In [3]:
@load "mystery_star.jld2"

1-element Vector{Symbol}:
 :atm

This is a `struct` of the type `Atmosphere3D` (check `tools.jl`), and has the following fields, which should be self-explanatory:

In [4]:
fieldnames(typeof(atm))

(:nx, :ny, :nz, :x, :y, :z, :temperature, :velocity_x, :velocity_y, :velocity_z, :electron_density, :hydrogen1_density, :proton_density)

*Please do not upload this model when you deliver you project*.

As defined in `tools.jl`, you can select an (i, j) index of this object and it will return an `Atmosphere1D` object, which can help simplify your code. E.g.:

In [5]:
atm11 = atm[1,1]
typeof(atm11)

Atmosphere1D

### 2.2 1D FALC model of the Sun

You will use the standard one-dimensional (1D) FALC model by 
[Fontenla et al. (1993)](http://adsabs.harvard.edu/abs/1993ApJ...406..319F).
The authors derived this model semi-empirically, assuming that the solar atmosphere is horizontally homogeneous ("plane parallel layers") and in hydrostatic equilibrium ("time independent"). This model was not derived from first-principle physics, but by a combination of physics and empirical adjustments (so called semi-empirical model), so that its properties match the observations of the Sun (as much as possible). 

<img src="https://tiagopereira.space/ast4310_h21/images/avrett_pic.svg" width=250px>

*Eugene H. Avrett (born 1933)
    represents the A in the VAL (Vernazza, Avrett and Loeser) and FAL
    (Fontenla, Avrett and Loeser) sequences of standard models of the
    solar atmosphere.
    At the Center for Astrophysics (Cambridge, Mass.) he developed,
    together with programmer Rudolf Loeser, an enormous spectrum
    modeling code (called Pandora, perhaps a fitting name) which fits
    observed solar continua and lines throughout the spectrum through
    the combination of trial-and-error adjustment of the temperature
    stratification with simultaneous evaluation of the corresponding
    particle densities with great sophistication, basically performing
    complete NLTE population analysis for all key transitions in the
    solar spectrum.
    His [VALIII paper](http://adsabs.harvard.edu/abs/1981ApJS...45..635V) stands as one of the three prime solar physics
    papers of the second half of the twentieth century (with [Parker's solar wind](http://adsabs.harvard.edu/abs/1958ApJ...128..664P) prediction and [Ulrich's p-mode](http://adsabs.harvard.edu/abs/1970ApJ...162..993U) prediction).
    Photograph from the web.*

This model is also provided as a JLD2 file, as a `struct` of type `Atmosphere1D`, which will load into a variable `falc` with the following command:

In [6]:
@load "falc.jld2"

1-element Vector{Symbol}:
 :falc

In [7]:
fieldnames(typeof(falc))

(:nz, :z, :temperature, :velocity_z, :velocity_turb, :electron_density, :hydrogen1_density, :proton_density)

Compared to `Atmosphere3D`. this structure has an additional field: `velocity_turb`. This is the microturbulence, often a fudge factor in 1D atmospheres to be quadratically added into the Doppler broadening ΔλD. The FALC model has values for `velocity_turb`, while the 3D model does not.

## 3. Synthetic Spectra


### 3.1 Solving the Radiative Transfer Equation

When we have a model atmosphere and calculate the radiation coming out of it, we call these synthetic spectra (as opposed to observed spectra). To obtain the synthetic spectrum of a model, we need to solve the radiative transfer throughout the atmosphere. In this project, you will calculate only the emergent radiation ($I_\lambda^+$ or $\mathcal{F}_\lambda^+$) by using the formal solution of the radiative transfer equation. Assuming a plane-parallel stratification, it can be written as:

\begin{equation}
  I_\lambda = \int_0^\infty S_\lambda e^{-\tau_\lambda}
              \rm{d} \tau_\lambda.
\end{equation}

In this project we are assuming LTE, so the source function is simply $S_\lambda=B_\lambda(T)$. The main unknown is therefore the extinction coefficient $\alpha_\lambda$. Once we have the extinction coefficient as function of height, we may compute the corresponding optical depth scale given by:

\begin{equation}
   \tag{1}
  \tau_\lambda(h_0) 
  \equiv -\int_\infty^{h_0} \alpha_\lambda \rm{d} h
\end{equation}

at any height $h_0$.  Note that both the model atmospheres (1D and 3D) are tabulated in reverse order, corresponding to the $-h$ direction (ie, depth). For the numerical $\tau$ integration, a simple approach is trapezoidal integration. In this particular case, we want a value of $\tau$ for each height point, so for every point we integrate $\alpha_\lambda$ *up to that point*, and not across the whole range of heights. This is called a *cumulative* integration, and is similar to what we did previously in Project 1. In general, this type of integration is numerically unstable for radiative transfer, but since here we only want the surface radiation, it is good enough.

The total extinction coefficient $\alpha_\lambda$ is the sum of continuum and line processes:

$$
\alpha_\lambda = \alpha_\lambda^c +\alpha_\lambda^l.
$$

### 3.2 Intensity from different directions of a 1D atmosphere

The disk-centre intensity can be calculated by using the integral expression for $S_\lambda \mathrm{e}^{-\tau_\lambda}$, which you can integrate with trapezoidal integration (e.g. from the package `NumericalIntegration.jl`). For the general case with different inclinations (given by $\mu=\cos\theta$) in a 1D plane-parallel atmosphere, the emergent intensity is given by

\begin{equation}
    I_\lambda(0, \mu) = \int_0^\infty S_\lambda \mathrm{e}^{-\tau_\lambda/\mu}d\tau_\lambda/\mu.
\end{equation}

We transform $\tau_\lambda \to \tau_\lambda/\mu$, meaning that inclined rays go through more material to reach the same depth in the z axis, and this increase in path length also increases the optical depth. This expression can be used to compute the intensity for any inclination (in the 1D case), which we can then use to calculate the surface flux $\cal{F}_\lambda$:

\begin{equation}
    \mathcal{F}_\lambda = \int I_\lambda \mathrm{d}\Omega  = 2\pi\int_{0}^1\mu I_\lambda(\mu) \mathrm{d}\mu.
\end{equation}

Numerically, the integral above needs to be discretized in a number of directions given by $\mu_i$, A problem arises that $I_\lambda$ cannot be evaluated at $\mu=0$, so this discretization is often done with a non-equidistant [Gauss-Legendre quadrature](https://en.wikipedia.org/wiki/Gauss%E2%80%93Legendre_quadrature). For this project, you can use a number of rays (points) n=5 from this quadrature, and since the points and weights are often tabulated for the [-1, 1] integral, you need to rescale the points and weights for the [0,1] integral:

\begin{eqnarray}
    x' &\to& \frac{x+1}{2}\\
    w' &\to& \frac{w}{2}
\end{eqnarray}

You can either hardcode these quadrature values, or use a library (e.g. FastGaussQuadrature.jl)

### 3.3 Continuum Extinction

<img src="https://upload.wikimedia.org/wikipedia/commons/6/67/Subrahmanyan_Chandrasekhar.gif" width=200px>

*Subrahmanyan Chandrasekhar (1910-1995) is regarded by many as the greatest astronomer of the twentieth century. He also played an important role in the topic of this exercise, undertaking a very lengthy and difficult calculation (not a computation, no computers yet at that time!) of the bound-free and free-free extinction cross-sections of the H$^-$ ion, together with F.H. Breen. Their result, published in [Chandrasekhar & Breen (1946)](http://adsabs.harvard.edu/abs/1946ApJ...104..430C) will be duplicated in one of your graphs below. The shape of the extinction curve, 
with a peak near $\lambda$ = 1 μm and a minimum at the H$^-$ bound-free ionisation limit at $\lambda$ = 1.6 μm , reproduced the observed spectral variation of the unknown solar continuum extinction as derived already by G.F.W. Mulders at Utrecht in 1935 for $\lambda$ = 0.4- 2.5 μm. The brilliant suggestion that H$^-$ might cause this extinction, instead of the ensemble of unknown metal edges that was earlier postulated, had already been given by [Pannekoek (1930)](http://adsabs.harvard.edu/abs/1930MNRAS..91..139P), but was forgotten until it came independently from [Wildt (1939)](http://adsabs.harvard.edu/abs/1939ApJ....89..295W). The proof came with Chandrasekhar's laborious quantum mechanical evaluation and explained how the solar atmosphere can be opaque at gas densities much lower than the transparent air around us, as you found in the previous exercise. It revolutionised the understanding of cool-star atmospheres by much increasing their hydrogen-to-metals ratio and gas pressure compared with the earlier metallic-absorption modeling of Biermann, Uns&ouml;ld and Pannekoek himself. A more detailed description is found in [Hearnshaw (1986)](http://adsabs.harvard.edu/abs/1986asoh.book.....H). Photograph from Wikimedia commons.*



<img src="https://tiagopereira.space/ast4310_h21/images/gray8-6a.svg" width=700px>

*Continuous extinction coefficients $\kappa_\nu^c$ for hydrogen and helium, per neutral hydrogen atom and per unit electron pressure, for the depth $\tau_0$ = 1 (continuum optical depth at $\lambda$ = 500 nm) in the photosphere of a solar-like dwarf star. The coefficients $\kappa$ are here measured per neutral hydrogen atom in whatever state of excitation, assuming Saha-Boltzmann population partitioning, and normalised by the electron pressure $P_\mathrm{e} = n_\mathrm{e} k \, T$. The cross-sections are in units of 10$^{-26}$cm$^2$ (not cm$^2$ as specified in the $y$-axis label); stimulated emission was not included in the computation of these curves. The H$^-$ curve shows the bound-free Balmer, Paschen and Brackett edges, plus part of the Pfund edge at right. The curves do not extend beyond the Balmer edge at left where the neglected metal edges become important. Thomson scattering is also neglected. From page 140 of [Gray (1992)](http://adsabs.harvard.edu/abs/1992oasp.book.....G).*


We will assume that H$^-$(a hydrogen atom with an extra electron) is the major provider of continuous extinction.  This is quite a good assumption
for the solar photosphere for wavelengths $\lambda$ > 0.5 μm. The second-best extinction provider are H I bound-free interactions, at only a few percent.  This may be seen in Figure above, taken from [Gray (1992)](http://adsabs.harvard.edu/abs/1992oasp.book.....G).

Below $\lambda$ = 500 nm there is heavy line crowding (not added in the figure from Gray) which acts as a quasi-continuum.  Below $\lambda$ = 365 nm the Balmer bound-free edge provides large extinction, and at yet shorter wavelengths the bound-free ionisaton edges of various metals (Al I, Mg I, Fe I, Si I, C I) provide steep extinction increase yet before the H I Lyman continuum sets in, as may be expected from the ionisation edges figure. In this exercise we will neglect these contributions by evaluating only the H$^-$ extinction and the extinction due to scattering off free electrons (Thomson scattering).

To calculate the H$^-$ continuous extinction we will use different tabulated values for the cross section of the H$^-$ bound-free and free-free contribution. For bound-free we use a table based on [Wishart (1979)](https://ui.adsabs.harvard.edu/abs/1979MNRAS.187P..59W) and [Broad and Reinhardt (1976)](https://ui.adsabs.harvard.edu/abs/1976PhRvA..14.2159B) (often referred to as WBR), coded in the function `α_hminus_bf()`. For free-free we use a table based on [Stilley & Callaway (1970)](https://ui.adsabs.harvard.edu/abs/1970ApJ...160..245S/abstract), coded in the function `α_hminus_ff()`. 

Two other continuum processes you will need to add are Thomson scattering from electrons (`α_thomson()`) and Rayleigh scattering from neutral H atoms (`α_rayleigh_h()`). 

Check `tools.jl` for details and calling sequence of this functions.

### 3.4 Line Extinction

<img src="https://tiagopereira.space/ast4310_h21/images/unsold.svg">

*Albrecht O.J. Uns&ouml;ld (1905-1995) was the first, in 1941 at Kiel, to study stellar composition ("abundance analysis")  in the detail permitted by proper physical understanding of line formation (for the B0 dwarf $\tau$ Scorpii, using spectrograms taken during a six-month visit to the US just before the second world war).  He used the so-called "coarse" analysis based on Minnaert's curve of growth for a Schuster-Schwarzschild single "reversing layer".  Later, he led a school of astrophysicists at Kiel in "fine analysis", combining LTE line formation with non-grey stratified atmosphere modeling just as you do in this project.  The technique was developed by L.H. Aller, C. de Jager and others in the fifties, and has dominated stellar abundance analysis throughout the second half of the twentieth century. He spelled out its physical basis in great detail in his ["Physik der Sternatmosph&auml;ren"](http://adsabs.harvard.edu/abs/1955psmb.book.....U). Portrait copied from [Hearnshaw (1986)](http://adsabs.harvard.edu/abs/1986asoh.book.....H).*



We now have almost all the necessary ingredients to calculate synthetic spectra for a model atmosphere. The last remaining ingredient is the line extinction coefficient, $\alpha_\lambda^l$. The monochromatic line extinction per m path length for a bound-bound transition between a lower level $l$  and an upper level $u$ is given by (in SI units, see Rutten IART page 62):
\begin{equation*}
\alpha_\nu^l = \frac{e^2}{4\varepsilon_0 m_ec}n_l f_{lu} \varphi(\nu-\nu_0)\left[1 - \mathrm{e}^{-hc/\lambda k_B T}\right].
\end{equation*}

For the line profile we will assume a Voigt profile in wavelength units, $V(a, \lambda)\equiv H(a, u)/(\sqrt{\pi}\Delta\lambda_D)$, with the additional $\lambda^2 / c$ to convert from units per frequency to per wavelength, so the extinction becomes

\begin{equation*}
\alpha_\lambda^l = \frac{e^2}{4\varepsilon_0 m_ec}n_l f_{lu}\frac{\lambda^2}{c}\frac{H(a, u)}{\sqrt{\pi}\Delta\lambda_D}\left[1 - \mathrm{e}^{-hc/\lambda k_B T}\right].
\end{equation*}


The two main unknowns above are the lower level populations $n_l$ and the line profile $H(a, u)$, which we address below.


#### 3.4.1 Level populations

The lower level populations $n_l$ are total number density of atoms in level $l$. An atomic level is related to $r$, a given ionisation stage (e.g. for a Na I line, $r=0$, for Na II, $r=1$, and so on). To obtain $n_l$, we first need to know $N_E$, the number density of atoms from element E, then how many of those are in ionisation stage $r$, and finally of those how many are in level $l$. $n_E$ is usually known by its ratio to the hydrogen number density, $N_H$. We can decompose $n_l$ in quantities that are more readily available:

\begin{equation*}
n_l = \frac{n_l}{N_E}\frac{N_E}{N_H}N_H.
\end{equation*}

$N_H$ is usually known from the model atmosphere, and $N_X/N_H$ is the abundance of element X relative to hydrogen. To make matters slightly confusing, astronomers usually define the abundance of an element not as $N_X/N_H$ but as a log quantity called $\epsilon_X$ or $A_X$:

\begin{equation*}
A_X \equiv \log_{10}\left(\frac{N_X}{N_H}\right) + 12.
\end{equation*}

Replacing in $n_l$ above, we get:
\begin{equation*}
n_l = \frac{n_l}{N_X} 10^{(A_X -12)}  N_H.
\end{equation*}

Since we are in LTE, the quantity $n_l/N_E$ is given by the combined Saha and Boltzmann distributions

\begin{eqnarray*}
   U_r 
   &\equiv& \sum_s g_{r,s} e^{-\chi_{r,s}/k_B T}
              \\
   \frac{n_{l}}{N_r}
   &=& \frac{g_{l}}{U_r} e^{-\chi_{l}/k_B T}
             \\
   \frac{N_{r+1}}{N_r}
   &=& \frac{1}{N_\mathrm{e}} \frac{2U_{r+1}}{U_r}
       \left(\frac{2 \pi m_\mathrm{e} k_B T}{h^2}\right)^{3/2} 
       e^{-\chi_r/k_B T},
\end{eqnarray*}

so that
\begin{equation*}
\frac{n_l}{N_X} = \frac{n_l}{N_r} \frac{N_r}{N_X},
\end{equation*}

where $n_l/N_r$ is given by the Boltzmann equation and $N_r/N_X$ (where $N_X = \sum_r N_r$) can be obtained by multiple runs of the Saha distribution, to obtain the ratios for all necessary states. This was what you did in Project 1. However, in that `Atom()` class you calculated the populations for all levels in all stages, which is overkill, especially when you have large atoms (e.g. Fe). It is more handy to compute $n_l/N_r$ only for the level of interest (the lower level of the spectral line), and only run the Saha sum for the most important ionisation stages.

Using the expression above for $n_l/N_r$ we get an expression that depends on the partition function $U_r(T)$, the lower level energy (or potential) $\chi_l$, and the statistical weight of the lower level $g_l$. The latter one is often combined in the ratio with $f_{lu}$ in the expression for $\alpha_\lambda$, so that we get:

\begin{equation}
\alpha_\lambda^l = \frac{e^2}{4\varepsilon_0 m_ec} g_l f_{lu} \frac{e^{-\chi_{l}/k_B T}}{U_r}  \frac{N_r}{N_X} 10^{(A_X -12)} N_H \frac{\lambda^2}{c}\frac{H(a, u)}{\sqrt{\pi}\Delta\lambda_D}\left[1 - \mathrm{e}^{-hc/\lambda k_B T}\right].
\end{equation}

The term $g_l f_{lu}$ ("gf-value") is often given in log form, $\log_{10}(gf)$, and is a measure of the transition strength. To calculate the partition functions for Fe atoms, use the interpolants below

In [8]:
"""
Partition functions for Fe I, Fe II, Fe III,
based on data from Gray (2005), p. 515.
"""
θ = 2.0:-0.2:0.2
logU = [1.317 1.332 1.350 1.372 1.402 1.446 1.519 1.664 2.049 3.760
        1.504 1.525 1.549 1.575 1.604 1.638 1.682 1.749 1.881 2.307
        1.304 1.312 1.320 1.328 1.337 1.350 1.370 1.411 1.518 1.887]
Ttable = 5040u"K" ./ θ
const pfunction_FeI = extrapolate(interpolate(Ttable, 10 .^ logU[1, :], SteffenMonotonicInterpolation()), Flat())
const pfunction_FeII = extrapolate(interpolate(Ttable, 10 .^ logU[2, :], SteffenMonotonicInterpolation()), Flat())
const pfunction_FeIII = extrapolate(interpolate(Ttable, 10 .^ logU[3, :], SteffenMonotonicInterpolation()), Flat());

Each of these `pfunction_FeI`, etc., can be called as if it was a function, for a given temperature. For example:

In [9]:
pfunction_FeI(5000u"K")

27.784505923603387

#### 3.4.2 Line Profile and Broadening

The Voigt function  $H(a,u)$ describes the extinction profile shape and is defined by:

\begin{eqnarray*}
   H(a,u) &\equiv& \frac{a}{\pi} \int_{-\infty}^{+\infty}
            \frac{e^{-y^2}}{(u - y)^2 + a^2} \mathrm{d} y
                          \\[1ex]
   y &=& \frac{v_\mathrm{los}}{c} \frac{\lambda_0}{\Delta \lambda_\mathrm{D}}\\
   u &=& \frac{\lambda-\lambda_0}{\Delta \lambda_\mathrm{D}}   
                          \\
   a &=& \frac{\lambda^2}{4\pi c}\frac{\gamma}{\Delta \lambda_\mathrm{D}},
\end{eqnarray*}

where $v_\mathrm{los}$ is the velocity along the line of sight and $a$ the damping parameter. The quantity $u$ is sometimes called the dimensionless wavelength, and is the wavelength difference from $\lambda_0$ divided by the Doppler width. In other words, it is a wavelength scale in terms of Doppler widths. When computing a line profile numerically, we need to decide which wavelengths to cover: how many points, and how far from the line core. Extending the wavelength range too much, and we are mostly computing continuum, which is a waste of time. But having a too narrow range means that we will miss out on the line wings, so that is not good either. Determining the wavelength range in terms of Doppler widths is more flexible, since it is independent of the line strength and $\lambda_0$. 

The Doppler width $\Delta \lambda_\mathrm{D}$ is not only set by the thermal broadening but in the 1D case it includes also the microturbulent "fudge parameter"  $v_\mathrm{turb}$ through defining it as:

\begin{equation*}
   \Delta \lambda_\mathrm{D} 
      \equiv \frac{\lambda_0}{c} \sqrt{\frac{2k_B T}{m_X} + v_\mathrm{turb}^2},
\end{equation*}
where $m_X$ is the mass of the atom that causes the spectral line. To calculate the line profile, we recommend you use the `voigt()` function in `tools.jl`, as it is a faster implementation than the version we used earlier. We also provide functions `doppler_width()` for ΔλD and `damping()` for $a$.

The damping parameter $a$ contains the different types of line broadening in $\gamma$, often natural broadening, van der Waals broadening, and Stark broadening. Natural broadening is the sum of the Einstein coefficients for spontaneous de-excitation from the upper level of the transition to all levels below:

\begin{equation*}
   \gamma_\mathrm{rad} = \sum_{l<u}A_{ul}.
\end{equation*}

This quantity is usually tabulated in databases such as the [NIST Atomic Line Database](https://physics.nist.gov/PhysRefData/ASD/lines_form.html). Note that in the NIST database, 
$\gamma_\mathrm{rad}$ is listed as $A_{ki}$, or transition probability, in units of s$^{-1}$.


For the van der Waals broadening you will use the recipe of [Anstee & O'Mara (1995)](https://ui.adsabs.harvard.edu/abs/1995MNRAS.276..859A) (also known as ABO recipe). This is one of the most realistic recipes available to date, but is not available for all types of spectral lines. This recipe accounts for broadening from neutral hydrogen atoms:

\begin{equation*}
    \gamma_\mathrm{vdW} = 2\, a_0^2 \left(\frac{4}{\pi}\right)^{\alpha/2} \Gamma\left(\frac{4-\alpha}{2}\right)\bar{v}\,\sigma\, \left(\frac{\bar{v}}{10\;\mathrm{km/s}}\right)^{-\alpha}\; n_{\mathrm{HI}},
\end{equation*}


where $n_{\mathrm{HI}}$ is the number density of neutral hydrogen, $a_0$ is the Bohr radius, $\Gamma$ is the Gamma function, $\bar{v}$ is

\begin{equation*}
    \bar{v} = \left(\frac{8 k_B T}{\pi\mu_m} \right)^{1/2},
\end{equation*}

with $\mu_m$ the reduced mass of the two atoms, hydrogen and the line-producing element:

\begin{equation*}
    \mu_m = 1/\left(\frac{1}{m_\mathrm{H}} + \frac{1}{m_\mathrm{X}}\right).
\end{equation*}

The only remaining values are $\alpha$ and $\sigma$, which are tabulated in [Anstee & O'Mara (1995)](https://ui.adsabs.harvard.edu/abs/1995MNRAS.276..859A). To compute $\gamma_\mathrm{vdW}$ according to the ABO recipe, use the combination of functions `const_barklem()` and `γ_barklem()` (the alpha and sigma are parameters that vary for each line). Here's a random example:

In [10]:
alpha = 0.275
bconst = const_barklem(23.4 * m_u, alpha, 291.)
γvdW = γ_barklem(alpha, bconst, 5000u"K", 1e22u"m^-3")

1.6534441907852829e8 rad s⁻¹

Because $\gamma_\mathrm{rad}$ and $\gamma_\mathrm{vdW}$ are both associated with a Lorentzian profile, we can just sum them into $\gamma$ and replace that in the equation for the damping $a$:

\begin{equation*}
   \gamma = \gamma_\mathrm{rad} + \gamma_\mathrm{vdW}
\end{equation*}

Stark broadening is usually weaker for the lines we are considering here, so we will ignore it.

#### 3.4.3 Line Profile in 3D

Computing line broadening in 3D is done in nearly the same way as in 1D, with the main difference being that the Doppler width does not need the added microturbulence, so we set $v_\mathrm{turb}$=0. The turbulent motions that $v_\mathrm{turb}$ was meant to emulate exist naturally in the 3D model. 

At each point in the 3D simulation the line profile has the usual Voigt profile shape $V(a, u)$ but it is shifted by $\Delta\lambda$, which is a function of $v_\mathrm{los}$ for each point in the grid, and each direction. But how do we enter this shift in the Voigt profile? 

The way the Voigt function is built, it is a function of the damping parameter $a$ and the dimensionless wavelength $u$:

$$
u \equiv \frac{\Delta\lambda_i}{\Delta\lambda_D}.
$$

Numerically, $\Delta\lambda_i$ is typically an array of wavelength shifts around the line centre $\lambda_0$, e.g. $\Delta\lambda_i = -0.2, -0.1, 0, 0.1, 0.2$ nm. For $u=$ 1 we are one Doppler width away from the line core, $u=$ 5, five Doppler widths and so on. By definition, the peak of the Voigt function will be at $u=$ 0. However, when there is a Doppler shift $\Delta\lambda_\mathrm{los}$ caused by a line-of-sight velocity we have:

$$
\Delta\lambda_\mathrm{los} = \lambda_0\frac{v_\mathrm{los}}{c},
$$

and the peak of the Voigt function will be moved to $u=-\Delta\lambda_\mathrm{los} / \Delta\lambda_D$. In this definition, a positive $v_\mathrm{los}$ corresponds to an upflow (movement towards the observer), meaning that the wavelength shift is negative (blueshift). The way to add this shift $\Delta\lambda_\mathrm{los}$ is *not* in the Doppler width, but in $u$, and at every grid point of the simulation, because the line of sight will be different at every grid point. In the 3D case, the calculation of the normalised velocity $u$ (for the Voigt function) should therefore be:

$$
u = \frac{\left(\Delta\lambda_i + \Delta\lambda_\mathrm{los} \right)}{\Delta\lambda_D}.
$$

When $\mu=1$, $v_\mathrm{los} = v_z$. For $\mu\neq 1$, you will need to project the velocities, so that

$$
v_\mathrm{los} = v_z \cos\theta + v_y \sin\theta \sin\varphi + v_x \sin\theta \cos\varphi,
$$

where $\theta$ is the polar angle ($\mu=\cos\theta$) and $\varphi$ the azimuthal angle.

To calculate the line profile in 3D with velocities, you will need to modify your functions to compute a line profile to include the necessary changes in $u$.